In [4]:
%matplotlib inline
import bagpy
from bagpy import bagreader
import rosbag
import pandas as pd

In [2]:
b = bagreader('../Data/A_Spot_Library_Dobie_Wed_Nov_10_57.bag')

[INFO]  Successfully created the data folder ../Data/A_Spot_Library_Dobie_Wed_Nov_10_57.


In [3]:
b.topic_table

,Topics,Types,Message Count,Frequency
0,/image_raw/compressed,sensor_msgs/CompressedImage,6687,29.939640
1,/joint_states,sensor_msgs/JointState,3696,16.440965
2,/joystick,sensor_msgs/Joy,13413,59.996195
3,/localization,amrl_msgs/Localization2DMsg,3698,16.562631
4,/navigation/cmd_vel,geometry_msgs/Twist,8990,40.001755
5,/odom,nav_msgs/Odometry,3696,16.546622
6,/scan,sensor_msgs/LaserScan,2228,9.915097
7,/spot/camera/back/camera_info,sensor_msgs/CameraInfo,1006,4.476450
8,/spot/camera/back/image/compressed,sensor_msgs/CompressedImage,1006,4.476875
9,/spot/camera/frontleft/camera_info,sensor_msgs/CameraInfo,1006,4.473008


In [6]:
bag = rosbag.Bag('../Data/A_Spot_Library_Dobie_Wed_Nov_10_57.bag')

In [7]:
def point_cloud_2_birdseye(points, res=0.125, side_range=(-16., 16.), fwd_range=(0, 32.)):
    x_points = points['x']
    y_points = points['y']
    z_points = points['z']
    
    # FILTER - To return only indices of points within desired cube
    # Three filters for: Front-to-back, side-to-side, and height ranges
    # Note left side is positive y axis in LIDAR coordinates
    f_filt = np.logical_and((x_points > fwd_range[0]), (x_points < fwd_range[1]))
    s_filt = np.logical_and((y_points > -side_range[1]), (y_points < -side_range[0]))
    filter = np.logical_and(f_filt, s_filt)
    indices = np.argwhere(filter).flatten()

    # KEEPERS
    x_points = x_points[indices]
    y_points = y_points[indices]
    z_points = z_points[indices]

    # CONVERT TO PIXEL POSITION VALUES - Based on resolution
    x_img = (-y_points / res).astype(np.int32)  # x axis is -y in LIDAR
    y_img = (-x_points / res).astype(np.int32)  # y axis is -x in LIDAR

    # SHIFT PIXELS TO HAVE MINIMUM BE (0,0)
    # floor & ceil used to prevent anything being rounded to below 0 after shift
    x_img -= int(np.floor(side_range[0] / res))
    y_img += int(np.ceil(fwd_range[1] / res))
    
    """
    # CLIP HEIGHT VALUES - to between min and max heights
    pixel_values = np.clip(a=z_points,
                           a_min=height_range[0],
                           a_max=height_range[1])

    # RESCALE THE HEIGHT VALUES - to be between the range 0-255
    pixel_values = scale_to_255(pixel_values,
                                min=height_range[0],
                                max=height_range[1])
    """
    pixel_values = 1

    # INITIALIZE EMPTY ARRAY - of the dimensions we want
    x_max = 1 + int((side_range[1] - side_range[0]) / res)
    y_max = 1 + int((fwd_range[1] - fwd_range[0]) / res)
    im = np.zeros([y_max, x_max], dtype=np.uint8)

    # FILL PIXEL VALUES IN IMAGE ARRAY
    im[y_img, x_img] = pixel_values

    return im

In [ ]:
bev_frms = []
for topic, msg, t in bag.read_messages(topic='/velodyne_points'):
    a = 